# setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

sklearn version:  1.0.2
TF version:  2.7.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


# Get the data

In [6]:
#작성자 전민재
import csv
def load_Emist(exsitNumpy=False):
    if (exsitNumpy == False):
        #"./emnist-byclass-test.csv"
        #train-set
    
        csv_train_data_file = open("../공용/datasets/Emnist/train_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        csv_train_label_file = open("../공용/datasets/Emnist/train_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
    
        
        f_train = csv.reader(csv_train_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        l_train=csv.reader(csv_train_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        #valid-set
    
        csv_valid_data_file = open("../공용/datasets/Emnist/valid_data.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        csv_valid_label_file = open("../공용/datasets/Emnist/valid_label.csv", "r", encoding="ms932", errors="", newline="" ) #리스트 형식 
        
        f_valid = csv.reader(csv_valid_data_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        l_valid = csv.reader(csv_valid_label_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
        X_train=[]
        y_train=[]
        X_valid=[]
        y_valid=[]
        for i, row in enumerate(f_train):
            #행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx]=int(char)
            #train
            #data 추가    
            X_train.append(row)
            
        for i, row in enumerate(f_valid):
            #행마다 int로 형변환
            for idx, char in enumerate(row):
                row[idx]=int(char)
            #valid
            #data 추가    
            X_valid.append(row)


        for i, row in enumerate(l_train):

            #train
            #label 추가
            y_train.append(int(row[0]))
            
        for i, row in enumerate(l_valid):

            #valid
            #label 추가
            y_valid.append(int(row[0]))
            
            
        

        #train
        X_train=np.array(X_train)
        X_train=X_train.reshape(-1,28,28)
        #valid
        X_valid=np.array(X_valid)
        X_valid=X_valid.reshape(-1,28,28)
        
        csv_train_data_file.close()
        csv_train_label_file.close()
        csv_valid_data_file.close()
        csv_valid_label_file.close()
        
        #kaggle dataset이 시계반대방향으로 90도 회전 되있고 상하 반전 되어있음
        def rotate_90(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[c][N-1-r] = m[r][c]
            return np.array(ret)

        def vreflect(m):
            N = len(m)
            ret = [[0] * N for _ in range(N)]

            for r in range(N):
                for c in range(N):
                    ret[r][c] = m[r][N-1-c]
            return np.array(ret)

        #train
        #회전
        for idx,i in enumerate(X_train):
            X_train[idx]=rotate_90(i)
        #상하반전
        for idx,i in enumerate(X_train):
            X_train[idx]=vreflect(i)

        #valid
        #회전
        for idx,i in enumerate(X_valid):
            X_valid[idx]=rotate_90(i)
        #상하반전
        for idx,i in enumerate(X_valid):
            X_valid[idx]=vreflect(i)
        
        
        #저장용
        #train
        #(533917,28,28)
        np.save('../공용/X_train',X_train)
        np.save('../공용/y_train',y_train)
        #valid
        np.save('../공용/X_valid',X_valid)
        np.save('../공용/y_valid',y_valid)
        
    elif exsitNumpy == True:
        
        X_train=np.load('../공용/X_train.npy')
        y_train=np.load('../공용/y_train.npy')
        X_valid=np.load('../공용/X_valid.npy')
        y_valid=np.load('../공용/y_valid.npy')
    
    return X_train, y_train, X_valid, y_valid


In [7]:
X_train, y_train, X_valid, y_valid = load_Emist(exsitNumpy=True)

In [8]:
#황성현
#data, batch size 성정
dataset_size=len(X_train)
batch_size=32


In [9]:
#황성현
X_train=np.reshape(X_train,[533917,784])
X_train.shape

(533917, 784)

In [10]:
#황성현
y_train=np.array(y_train)
y_train=np.reshape(y_train,[533917,1])
y_train.shape

(533917, 1)

In [11]:
#황성현
train_full=np.append(X_train,y_train,axis=1)
train_full.shape

(533917, 785)

In [12]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    Emnist_dir = os.path.join("../공용/datasets", "Emnist")
    os.makedirs(Emnist_dir, exist_ok=True)
    path_format = os.path.join(Emnist_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        try:
            with open(part_csv, "xt", encoding="utf-8") as f:
                if header is not None:
                    f.write(header)
                    f.write("\n")
                for row_idx in row_indices:
                    f.write(",".join([repr(col) for col in data[row_idx]]))
                    f.write("\n")
        except:
            continue
    return filepaths

In [13]:
train_filepaths = save_to_multiple_csv_files(train_full, "train", n_parts=20)

In [14]:
n_inputs = X_train.shape[-1]
print("n_inputs =",n_inputs)
@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    x=tf.reshape(x,[28,28,1])
    resized_image = tf.image.resize(x, [224, 224])
    final_image = keras.applications.mobilenet.preprocess_input(resized_image)
    y = tf.stack(fields[-1:])
    return final_image, y

n_inputs = 784


In [15]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [16]:
tf.random.set_seed(42)

train_set = csv_reader_dataset(train_filepaths,batch_size=batch_size, repeat=None)
train_set

<PrefetchDataset shapes: ((None, 224, 224, 1), (None, 1)), types: (tf.float32, tf.float32)>

# SMV2

In [27]:
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c*t,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c*t,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3,strides=s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=c,kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [28]:
n_classes=62
smv2_l=keras.models.Sequential([
    keras.layers.BatchNormalization(input_shape=[224,224,1]),
    keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
    Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
    Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
    Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
    Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
    Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
    Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
    keras.layers.ReLU(max_value=6),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(n_classes,activation="softmax")
])
smv2_l.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 224, 224, 1)      4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 112, 112, 16)     1216      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 56, 56, 16)       6688      
 k)                                                              
                                                                 
 IRB3 (Inveted_Residual_Bloc  (None, 28, 28, 32)       38656     
 k)                                                     

In [29]:
smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",metrics=["accuracy"])

In [30]:
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2P%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam2.h5",save_best_only=True)
history=smv2_l.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                   
                    epochs=10,callbacks=[])

Epoch 1/10
12513/12513 [==============================] - 2913s 232ms/step - loss: 0.5816 - accuracy: 0.8074
Epoch 2/10
12513/12513 [==============================] - 2927s 234ms/step - loss: 0.4530 - accuracy: 0.8390
Epoch 3/10
12513/12513 [==============================] - 2914s 233ms/step - loss: 0.4247 - accuracy: 0.8468
Epoch 4/10
12513/12513 [==============================] - 2921s 233ms/step - loss: 0.4192 - accuracy: 0.8488
Epoch 5/10
12513/12513 [==============================] - 2908s 232ms/step - loss: 0.4119 - accuracy: 0.8518
Epoch 6/10
 9775/12513 [======================>.......] - ETA: 10:36 - loss: 0.4100 - accuracy: 0.8516

KeyboardInterrupt: 